# self attention 알고리즘

In [1]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def self_attention(input_sequence):
    output = np.zeros(shape=input_sequence.shape) #  |3, 5|
    for i, pivot_vector in enumerate(input_sequence): # 3번 돌아감
        scores = np.zeros(shape=(len(input_sequence),)) # |3|
        
        # 각 input vector에 pivot_vector와의 dot product
        for j, vector in enumerate(input_sequence): # 3번 돌아감
            scores[j] = np.dot(pivot_vector, vector.T) # |1, 1|
        scores /= np.sqrt(input_sequence.shape[1]) # np.sqrt(5)
        scores = softmax(scores)
        
        new_pivot_representation = np.zeros(shape=pivot_vector.shape)
        for j, vector in enumerate(input_sequence):
            new_pivot_representation += vector * scores[j]
        output[i] = new_pivot_representation
    
    return output


In [17]:
input_sequence = np.random.rand(3, 5)
print("Input Sequence:")
print(input_sequence)
print(len(input_sequence))


Input Sequence:
[[0.16157119 0.73900811 0.65988113 0.4454785  0.49720242]
 [0.70731463 0.87360794 0.27799402 0.2553986  0.85631822]
 [0.84295323 0.5089968  0.30807629 0.39465432 0.56764531]]
3


In [40]:
attention_output = self_attention(input_sequence)
print("\nSelf-Attention Output:")
print(attention_output)


Self-Attention Output:
[[0.56018399 0.71601487 0.41904062 0.36347656 0.64433295]
 [0.59270694 0.71742156 0.39835956 0.355248   0.65945404]
 [0.59557001 0.71006672 0.39888733 0.35802747 0.65368484]]


In [16]:
np.zeros(shape=(5,))

array([0., 0., 0., 0., 0.])

In [10]:
np.dot(input_sequence[1], input_sequence[1].T)

1.9770226759738583

In [12]:
np.dot(input_sequence[1], input_sequence[1])

1.9770226759738583

# layer_normalization, batch_normalization

In [19]:
def layer_normalization(batch_of_sequences):
    mean = np.mean(batch_of_sequences, axis=-1, keepdims=True) # (2,2,1)
    variance = np.var(batch_of_sequences, axis=-1, keepdims=True) # (2,2,1)
    return (batch_of_sequences - mean) / np.sqrt(variance + 1e-6)

def test_layer_normalization():
    # 임의의 시퀀스 배치 생성
    batch_of_sequences = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]], dtype=np.float32) # (2,2,2)
    
    # 레이어 정규화 실행
    normalized_batch = layer_normalization(batch_of_sequences) # 
    
    # 기대 결과 계산
    mean = np.mean(batch_of_sequences, axis=-1, keepdims=True)
    variance = np.var(batch_of_sequences, axis=-1, keepdims=True)
    expected_output = (batch_of_sequences - mean) / np.sqrt(variance + 1e-6)
    
    # 결과 비교
    assert np.allclose(normalized_batch, expected_output), "layer_normalization function does not perform as expected"

# 함수 실행
test_layer_normalization()


In [20]:
def batch_normalization(batch_of_images): # (2, 2, 2, 3)
    mean = np.mean(batch_of_images, axis=(0, 1, 2), keepdims=True) # (1, 1, 1, 3)
    variance = np.var(batch_of_images, axis=(0, 1, 2), keepdims=True) # (1, 1, 1, 3)
    return (batch_of_images - mean) / np.sqrt(variance + 1e-6)

def test_batch_normalization():
    # 임의의 이미지 배치 생성 (예: 배치 크기=2, 높이=2, 너비=2, 채널=3)
    batch_of_images = np.array([[[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]],
                                [[[13, 14, 15], [16, 17, 18]], [[19, 20, 21], [22, 23, 24]]]], dtype=np.float32) # (2, 2, 2, 3)
    
    # 배치 정규화 실행
    normalized_batch = batch_normalization(batch_of_images)
    
    # 기대 결과 계산
    mean = np.mean(batch_of_images, axis=(0, 1, 2), keepdims=True)
    variance = np.var(batch_of_images, axis=(0, 1, 2), keepdims=True)
    expected_output = (batch_of_images - mean) / np.sqrt(variance + 1e-6)
    
    # 결과 비교
    assert np.allclose(normalized_batch, expected_output), "batch_normalization function does not perform as expected"

# 함수 실행
test_batch_normalization()


`BatchNormalization`과 `LayerNormalization`은 둘 다 데이터를 정규화하여 딥러닝 모델의 학습을 돕지만, 이들이 데이터를 처리하는 방식에는 중요한 차이점이 있습니다.

### BatchNormalization

`BatchNormalization`은 여러 샘플(데이터 포인트)에서 정보를 수집하여, 특성(feature)의 평균과 분산의 정확한 통계를 얻습니다. 즉, 배치 내의 모든 샘플을 통틀어 평균과 분산을 계산합니다. 이 방법은 이미지 처리 같은 분야에서 매우 효과적이며, 모델이 더 빠르게 학습하고 더 좋은 성능을 내도록 도울 수 있습니다. 예를 들어, CNN(Convolutional Neural Network)에서 이미지의 배치를 처리할 때, 전체 배치에 대한 평균 및 분산을 사용하여 각 이미지를 정규화합니다.

### LayerNormalization

반면, `LayerNormalization`은 각 시퀀스 내에서 별도로 데이터를 풀링하여 정규화를 수행합니다. 이는 주로 시퀀스 데이터(예: 텍스트 또는 시계열 데이터)에 적합한 방식입니다. `LayerNormalization`은 시퀀스의 각 요소(예: 각 단어 또는 각 시간 단계)에 대해 동일한 평균과 분산을 사용하여 정규화를 진행합니다. 이 방법은 시퀀스 각각의 내부 구조를 유지하면서 각 시퀀스의 특성을 더 잘 반영할 수 있게 합니다. 예를 들어, RNN(Recurrent Neural Network)이나 Transformer 같은 모델에서 텍스트 처리를 할 때 각 문장(시퀀스) 내에서 정규화를 진행합니다.

### 쉬운 설명

상상해 보세요, 여러분이 다양한 국가에서 온 여러 그룹의 사람들의 키를 조사한다고 합니다. `BatchNormalization`은 모든 국가의 사람들을 통틀어 평균 키와 키의 분산을 계산합니다. 이는 전체적으로 어느 정도의 일관성을 제공하지만, 각 국가별 특성은 고려하지 않습니다.

반면에, `LayerNormalization`은 각 국가의 사람들을 개별적으로 봅니다. 각 국가 내에서 사람들의 키에 대해 평균과 분산을 계산합니다. 이렇게 하면 각 국가의 특성을 더 잘 반영할 수 있으며, 그룹 내에서 더 유의미한 정규화를 수행할 수 있습니다.

`BatchNormalization`은 전체 데이터셋의 특성을 고르게 반영하는 반면, `LayerNormalization`은 개별 시퀀스나 데이터 포인트의 내부 구조와 특성을 더 잘 살려서 정규화하는 것입니다.

## BatchNormalization layer가 sequence data에는 잘 작동하지 않는 이유

`BatchNormalization`이 시퀀스 데이터에 잘 작동하지 않는 주요 이유는 시퀀스 데이터의 특성과 `BatchNormalization`의 작동 방식 사이에 존재하는 근본적인 차이 때문입니다. 시퀀스 데이터, 특히 자연어 처리(NLP)나 시계열 데이터는 다음과 같은 특징을 가지고 있습니다:

1. **시퀀스의 길이가 다양함**: 시퀀스 데이터는 각 시퀀스(예: 문장, 시계열 데이터 포인트)의 길이가 서로 다를 수 있습니다. 이는 `BatchNormalization`을 적용할 때 문제가 될 수 있는데, `BatchNormalization`은 배치 내의 모든 데이터 포인트에 대해 동일한 평균과 분산을 계산하고 적용합니다. 길이가 다른 시퀀스를 같은 방식으로 정규화하려고 하면, 각 시퀀스의 고유한 맥락과 구조가 손상될 수 있습니다.

2. **시퀀스 내 위치별 정보의 중요성**: 시퀀스 데이터에서는 각 요소(예: 단어, 시간 단계)의 위치가 중요합니다. `BatchNormalization`은 위치별 정보를 고려하지 않고 전체 배치에 걸쳐 동일한 통계(평균과 분산)를 적용하기 때문에, 시퀀스 내에서 각 위치의 고유한 정보를 충분히 반영하지 못할 수 있습니다.

3. **시퀀스 데이터의 동적 특성**: 특히 시계열 데이터 같은 경우, 시간에 따라 데이터의 특성이 변할 수 있습니다. 이러한 동적 특성은 `BatchNormalization`이 고정된 통계를 사용하여 정규화를 수행하는 방식과 잘 맞지 않을 수 있습니다.

4. **배치 내 변이성**: 시퀀스 데이터를 처리할 때, 배치 내에 다양한 시퀀스가 포함될 수 있습니다. 이러한 다양성은 `BatchNormalization`이 평균과 분산을 계산할 때 각 시퀀스의 고유한 특성을 무시하게 만들 수 있습니다. 결과적으로, 특정 시퀀스에 대한 정규화가 적절하지 않을 수 있습니다.

따라서, 시퀀스 데이터와 같이 내부 구조와 맥락이 중요한 경우, `LayerNormalization` 같은 다른 정규화 기법이 더 적합할 수 있습니다. `LayerNormalization`은 각 시퀀스 내에서 독립적으로 평균과 분산을 계산하고 적용하기 때문에, 시퀀스의 길이, 구조, 위치별 정보 등을 더 잘 보존할 수 있습니다.

# 신경망이 큰 입력 값이나 이산적인 입력 분포에는 잘 동작하지 않는 이유는?

신경망이 큰 입력 값이나 이산적인 입력 분포에 잘 동작하지 않는 주요 이유는 다음과 같은 신경망의 특성과 제약 때문입니다:

### 1. **그라디언트 소실 또는 폭발**

- **큰 입력 값**: 신경망의 활성화 함수(예: 시그모이드, 탄젠트)는 입력 값의 범위가 넓을 경우 그라디언트 소실 문제를 일으킬 수 있습니다. 입력 값이 너무 크면, 활성화 함수의 출력이 포화 상태에 도달하여 그라디언트가 매우 작아지거나 0에 가까워집니다. 이는 학습 과정에서 가중치 업데이트가 제대로 이루어지지 않게 만들어 학습이 잘 진행되지 않을 수 있습니다.
- **그라디언트 폭발**: 반대로, 너무 큰 입력 값은 계산 과정에서 그라디언트의 크기가 급격히 커져서 그라디언트 폭발 문제를 일으킬 수 있습니다. 이는 모델 학습을 불안정하게 만들 수 있습니다.

### 2. **일반화 능력 저하**

- **이산적인 입력 분포**: 신경망은 입력 데이터의 패턴을 학습하여 일반화하는 데 목적이 있습니다. 이산적인 입력 분포는 특정 값 또는 카테고리에 집중되어 있어, 모델이 데이터의 내재된 연속적인 특성을 학습하기 어렵게 만들 수 있습니다. 이로 인해 모델의 일반화 능력이 저하되고, 새로운 데이터나 미세하게 다른 데이터에 대해 예측하는 능력이 떨어질 수 있습니다.

### 3. **학습의 비효율성**

- **큰 값과 작은 값의 비율**: 입력 데이터에 큰 값과 작은 값이 혼재되어 있을 경우, 신경망은 작은 값의 변화를 잘 감지하지 못할 수 있습니다. 이는 큰 값에 의해 그라디언트가 지배되어 작은 값의 중요성이 무시되기 때문입니다. 결과적으로 모델이 중요한 정보를 놓치고 학습이 비효율적으로 이루어질 수 있습니다.

### 해결 방안

- **데이터 정규화**: 입력 데이터를 정규화하여 모든 값을 비슷한 범위로 조정함으로써 그라디언트 소실이나 폭발 문제를 완화할 수 있습니다.
- **활성화 함수 선택**: ReLU(렉티파이드 리니어 유닛) 같은 활성화 함수는 큰 양수 입력 값에 대해 포화되지 않아 그라디언트 소실 문제를 어느 정도 방지할 수 있습니다.
- **모델 구조 조정**: 네트워크 구조를 조정하여 이산적인 입력 분포를 더 잘 처리할 수 있게 만들 수 있습니다. 예를 들어, 임베딩 레이어를 사용하여 이산적인 값을 연속적인 고차원 벡터로 변환할 수 있습니다.

신경망이 큰 입력 값이나 이산적인 입력 분포에 잘 동작하지 않는 이유는 이러한 값들이 신경망의 학습 과정, 특히 그라디언트 기반 학습에 여러 문제를

 일으키기 때문입니다. 이를 해결하기 위한 다양한 기술과 방법이 개발되고 있습니다.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

2024-04-11 14:51:36.060154: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-11 14:51:36.060178: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

2024-04-11 14:51:40.850709: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-11 14:51:40.850822: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-04-11 14:51:40.850901: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-04-11 14:51:40.850975: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2024-04-11 14:51:40.851048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

In [3]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs): # 1
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding( # 2
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim) # 3
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions # 4

    def compute_mask(self, inputs, mask=None): # 5
        return tf.math.not_equal(inputs, 0) # 5

    def get_config(self): # 6
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [4]:

vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        5273600   
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           